In [1]:
from agents import Agent
import os, glob, re
import numpy as np
import pandas as pd

In [2]:
init_habit_strength = np.array([[0.0, 0.57],
                                [0.0, 0.57],
                                [0.0, 0.57],
                                [0.0, 0.57],
                                [0.0, 0.57],
                                [0.57, 0.0],  
                                [0.57, 0.0]])
    
csv_files = glob.glob(os.path.join('//global/scratch/users/norahluo/trajectory', '*.csv'))
demo = pd.read_csv('//global/scratch/users/norahluo/demo.csv')

def initialize():
    agents = []
    for f in csv_files:
        df = pd.read_csv(f, parse_dates = ['date'], usecols = lambda x: 'Unnamed' not in x)
        df = df.loc[(df.date >= '2020-01-06')]
        df.reset_index(drop = True, inplace = True)
        demo_ = demo[demo.panelist_id == re.split(r'_|\.',f)[1]]        
        df['dfrw'] = demo_['dfrw_cap_mean'].values[0]           
        df['income'] = demo_['hh_income_mid'].values[0]
        df['college'] = 1 if demo_['edu_level'].values[0] in ['University/College', 'Postgraduate Education'] else 0
        constraint = []
        agent = Agent(init_habit_strength, df, param_constraint = constraint)
        agents.append(agent)
    return agents

In [3]:
def losses(params):
    losses = 0
    agents = initialize()
    for agent in agents:
        agent.params = params
        loss, _, _, _ = agent.compute_loss()
        losses += loss
    return losses

In [4]:
init_params = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/0.128-1), 0.3, 3.531/10, np.array([0.904, -0.009*10, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394]), 20.378, 1.014]))
losses = losses(init_params)

In [96]:
init_params['w0'] -= h

In [97]:
init_params

{'HDP': 1.9187591599893623,
 'HGP': 0.3,
 'hs': 0.3531,
 'B': array([ 0.904, -0.09 , -0.046, -0.455, -0.017,  0.071, -0.581, -1.394]),
 'wh': 20.378,
 'w0': 1.01399}

In [137]:

h = 1e-4
# init_params = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/0.128-1), 0.3, 3.531/10, np.array([0.904, -0.009*10, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394]), 20.378, 1.014]))
# losses_init = losses(init_params)

params_plus = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/(0.128+h)-1), 0.3, 3.531/10, np.array([0.904, -0.009*10, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394]), 20.378, 1.014]))
losses_plus = losses(params_plus)

params_minus = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/(0.128-h)-1), 0.3, 3.531/10, np.array([0.904, -0.009*10, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394]), 20.378, 1.014]))
losses_minus = losses(params_minus)

(losses_plus - 2*254500.64195170635 + losses_minus) / (h**2)

419614.17300626636

In [133]:
%time
numerical_second_derivative(losses, 'HDP', 1e-5)

313924.1016469895

In [132]:
def numerical_second_derivative(f, x, h):
    params = dict(zip(['HDP', 'hs', 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'wh', 'w0'], [0.128, 3.531, 0.904, -0.009, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394, 20.378, 1.014]))
    
    # init_params = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params['HDP']-1), 0.3, params['hs']/10, np.array([params['b'+str(i)]*10 if i == 1 else params['b'+str(i)] for i in range(8)]), params['wh'], params['w0']]))
    # losses_init = f(init_params)

    params_plus = params.copy()
    params_plus[x] += h
    params_plus_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_plus['HDP']-1), 0.3, params_plus['hs']/10, np.array([params_plus['b'+str(i)]*10 if i == 1 else params_plus['b'+str(i)] for i in range(8)]), params_plus['wh'], params_plus['w0']]))
    losses_plus = f(params_plus_)

    params_minus = params.copy()
    params_minus[x] -=h
    params_minus_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_minus['HDP']-1), 0.3, params_minus['hs']/10, np.array([params_minus['b'+str(i)]*10 if i == 1 else params_minus['b'+str(i)] for i in range(8)]), params_minus['wh'], params_minus['w0']]))
    losses_minus = f(params_minus_)

    fish = (losses_plus - 2*254500.64195170635 + losses_minus) / (h**2)

    return fish

In [116]:
def numerical_mixed_second_derivative(f, x1, x2, h):

    params = dict(zip(['HDP', 'hs', 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'wh', 'w0'], [0.128, 3.531, 0.904, -0.009, -0.046, -0.455, -0.017, 0.071, -0.581, -1.394, 20.378, 1.014]))
    
    params_plus = params.copy()
    params_plus[x1] += h
    params_plus[x2] += h
    params_plus_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_plus['HDP']-1), 0.3, params_plus['hs']/10, np.array([params_plus['b'+str(i)]*10 if i == 1 else params_plus['b'+str(i)] for i in range(8)]), params_plus['wh'], params_plus['w0']]))
    losses_plus = f(params_plus_)

    params_minus = params.copy()
    params_minus[x1] -= h
    params_minus[x2] -= h
    params_minus_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_minus['HDP']-1), 0.3, params_minus['hs']/10, np.array([params_minus['b'+str(i)]*10 if i == 1 else params_minus['b'+str(i)] for i in range(8)]), params_minus['wh'], params_minus['w0']]))
    losses_minus = f(params_minus_)

    params_mix1 = params.copy()
    params_mix1[x1] += h
    params_mix1[x2] -= h
    params_mix1_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_mix1['HDP']-1), 0.3, params_mix1['hs']/10, np.array([params_mix1['b'+str(i)]*10 if i == 1 else params_mix1['b'+str(i)] for i in range(8)]), params_mix1['wh'], params_mix1['w0']]))
    losses_mix1 = f(params_mix1_)

    params_mix2 = params.copy()
    params_mix2[x1] -= h
    params_mix2[x2] += h
    params_mix2_ = dict(zip(['HDP', 'HGP', 'hs', 'B', 'wh', 'w0'], [np.log(1/params_mix2['HDP']-1), 0.3, params_mix2['hs']/10, np.array([params_mix2['b'+str(i)]*10 if i == 1 else params_mix2['b'+str(i)] for i in range(8)]), params_mix2['wh'], params_mix2['w0']]))
    losses_mix2 = f(params_mix2_)

    fish = (losses_plus - losses_mix1 - losses_mix2 + losses_minus) / (h**4)

    return fish

In [110]:
np.linalg.inv(a)

array([[ 0.80555556, -0.47222222,  0.22222222],
       [-0.47222222,  0.13888889,  0.11111111],
       [ 0.22222222,  0.11111111, -0.11111111]])

In [120]:
cov = np.zeros((12,12))
vars = ['HDP', 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'wh', 'w0','hs']
for i in range(12):
    print(vars[i])
    cov[i,i] = numerical_second_derivative(losses, vars[i], h)
    for j in range(i+1, 12):
        print(vars[j])
        cov_ = numerical_mixed_second_derivative(losses, vars[i], vars[j], h)
        cov[i,j] = cov_
        cov[j,i] = cov_

HDP
b0
b1
b2
b3
b4
b5
b6
b7
wh
w0
hs
b0
b1
b2
b3
b4
b5
b6
b7
wh
w0
hs
b1
b2
b3
b4
b5
b6
b7
wh
w0
hs
b2
b3
b4
b5
b6
b7
wh
w0
hs
b3
b4
b5
b6
b7
wh
w0
hs
b4
b5
b6
b7
wh
w0
hs
b5
b6
b7
wh
w0
hs
b6
b7
wh
w0
hs
b7
wh
w0
hs
wh
w0
hs
w0
hs
hs


In [141]:
np.linalg.inv(cov)

array([[ 1.19201622e-16, -7.43117744e-17, -4.43476610e-18,
        -6.84554432e-18, -4.72388855e-15,  4.06644479e-15,
        -1.34956900e-15,  3.08574533e-15, -1.28850673e-14,
         1.35488498e-14,  8.30981239e-16, -9.31070467e-16],
       [-7.43117744e-17, -2.73431609e-15,  5.67181680e-17,
         9.39444776e-17,  3.61198572e-15, -3.92203294e-15,
         3.20286083e-15,  3.70768445e-15, -6.31772105e-16,
        -4.50613003e-15, -5.96260363e-16,  1.23433392e-15],
       [-4.43476610e-18,  5.67181680e-17, -1.05105696e-17,
         5.08841005e-18,  2.73774735e-16, -3.45160365e-17,
         7.45597297e-17,  2.04706646e-16,  4.06025610e-16,
        -6.07008222e-16, -5.92282044e-17,  3.02649559e-17],
       [-6.84554432e-18,  9.39444776e-17,  5.08841005e-18,
        -2.33469695e-17,  2.94080839e-16, -3.54523702e-16,
         3.00331815e-16,  3.44214799e-16, -3.76012190e-17,
        -3.76983938e-16, -4.63898451e-17,  1.12673901e-16],
       [-4.72388855e-15,  3.61198572e-15,  2.7377473